In [1]:
from mlflow.tracking import MlflowClient

In [2]:
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri = MLFLOW_TRACKING_URI)

In [3]:
# List all experiments
experiments = client.search_experiments()

# Print experiment details
for exp in experiments:
    print(f"Name: {exp.name}, ID: {exp.experiment_id}")

Name: my-experiment, ID: 2
Name: nyc-taxi-experiment, ID: 1


Create experiment

In [5]:
# client.create_experiment(name="my-experiment")

In [6]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [7]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: e235b6d6839342c190aa9c06f9ce5ad4, rmse: 6.3886
run id: b3368d9c338e423bb0c0fe7d7f8b078c, rmse: 6.4684
run id: b73e8c2a7c4344e6be28127a5686221d, rmse: 6.6021
run id: 60a8d1e896ec46fca9ff11aa76e2bd72, rmse: 9.2334
run id: affd243c4001422ba3648b81a5b998ea, rmse: 9.2334


#### Register a new version of the model

In [8]:
import mlflow
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [9]:
run_id="b73e8c2a7c4344e6be28127a5686221d"
model_uri=f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '4' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1716764070473, current_stage='None', description=None, last_updated_timestamp=1716764070473, name='nyc-taxi-regressor', run_id='b73e8c2a7c4344e6be28127a5686221d', run_link=None, source='/workspaces/mlops-zoomcamp/03-experiment-tracking/mlruns/1/b73e8c2a7c4344e6be28127a5686221d/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

In [10]:
model_name = "nyc-taxi-regressor"
versions = client.get_latest_versions(name=model_name)

for version in versions:
    print(f"version: {version.version} stage: {version.current_stage}")

version: 4 stage: None
version: 3 stage: Production


/tmp/ipykernel_33037/2291241426.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  versions = client.get_latest_versions(name=model_name)


In [11]:
client.transition_model_version_stage(
    name=model_name,
    version=3,
    stage="Staging",
    archive_existing_versions=False
)

/tmp/ipykernel_33037/3248838605.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1716762134257, current_stage='Staging', description=None, last_updated_timestamp=1716764071623, name='nyc-taxi-regressor', run_id='b73e8c2a7c4344e6be28127a5686221d', run_link=None, source='/workspaces/mlops-zoomcamp/03-experiment-tracking/mlruns/1/b73e8c2a7c4344e6be28127a5686221d/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

#### Download artifacts

In [12]:
run_id ="72200330cb6b4f9ab48bd948dcca3668"
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

MlflowException: The following failures occurred while downloading one or more artifacts from /workspaces/mlops-zoomcamp/03-experiment-tracking/mlruns/1/72200330cb6b4f9ab48bd948dcca3668/artifacts:
##### File preprocessor #####
[Errno 2] No such file or directory: '/workspaces/mlops-zoomcamp/03-experiment-tracking/mlruns/1/72200330cb6b4f9ab48bd948dcca3668/artifacts/preprocessor'

#### Load preprocessor

In [13]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv=pickle.load(f_in)

#### Transition model to production

In [14]:
client.transition_model_version_stage(
    name=model_name,
    version=3,
    stage="Production",
    archive_existing_versions=True
)

/tmp/ipykernel_33037/684772386.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1716762134257, current_stage='Production', description=None, last_updated_timestamp=1716764087937, name='nyc-taxi-regressor', run_id='b73e8c2a7c4344e6be28127a5686221d', run_link=None, source='/workspaces/mlops-zoomcamp/03-experiment-tracking/mlruns/1/b73e8c2a7c4344e6be28127a5686221d/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

#### Workaround 2.9 mlflow transition_model_version_stage deprecation

In [18]:
def assign_alias_to_stage(model_name, stage, alias):
    """
    Assign an alias to the latest version of a registered model within a specified stage.

    :param model_name: The name of the registered model.
    :param stage: The stage of the model version for which the alias is to be assigned. Can be
                  "Production", "Staging", "Archived", or "None".
    :param alias: The alias to assign to the model version.
    :return: None
    """
    #latest_mv = client.get_latest_versions(model_name, stages=[stage])[0]
    latest_mv = client.get_latest_versions(model_name)[0]
    client.set_registered_model_alias(model_name, alias, latest_mv.version)

assign_alias_to_stage(model_name=model_name, stage="Staging", alias="testing-mlflow")

/tmp/ipykernel_33037/2689145377.py:12: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  latest_mv = client.get_latest_versions(model_name)[0]


In [17]:
client.get_latest_versions(model_name)

/tmp/ipykernel_33037/3777187309.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  client.get_latest_versions(model_name)


[<ModelVersion: aliases=[], creation_timestamp=1716764070473, current_stage='None', description=None, last_updated_timestamp=1716764070473, name='nyc-taxi-regressor', run_id='b73e8c2a7c4344e6be28127a5686221d', run_link=None, source='/workspaces/mlops-zoomcamp/03-experiment-tracking/mlruns/1/b73e8c2a7c4344e6be28127a5686221d/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>,
 <ModelVersion: aliases=[], creation_timestamp=1716762134257, current_stage='Production', description=None, last_updated_timestamp=1716764087937, name='nyc-taxi-regressor', run_id='b73e8c2a7c4344e6be28127a5686221d', run_link=None, source='/workspaces/mlops-zoomcamp/03-experiment-tracking/mlruns/1/b73e8c2a7c4344e6be28127a5686221d/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>]